## 1. Setup Environment

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import logging

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2. Analyze Data
### 2.1. Check for missing values follow the column

In [ ]:
combined_df = pd.read_csv("../data/combined/combined_travistorrent.csv")
combined_df.isna().mean()

### 2.2. Check for missing values follow the row

In [ ]:
nan_per_row = combined_df.isna().sum(axis=1)

# Count frequency and calculate ratio
nan_counts = nan_per_row.value_counts().sort_index()
nan_ratios = nan_counts / len(combined_df)

# Create DataFrame for tabular display
table_df = pd.DataFrame({
    'NaN columns number': nan_counts.index,
    'Row number': nan_counts.values,
    'Ratio (%)': (nan_ratios * 100).values
})

# Làm tròn tỷ lệ
table_df['Ratio (%)'] = table_df['Ratio (%)'].round(2)

# In bảng
print("\nTable of ratio of rows by number of NaN columns:")
print(table_df)

# Vẽ biểu đồ tròn
plt.figure(figsize=(3, 3))
plt.pie(table_df['Ratio (%)'], labels=table_df['NaN columns number'], autopct='%1.1f%%', startangle=140,
        colors=plt.cm.Paired.colors)
plt.title('Ratio of rows to number of columns NaN')
plt.axis('equal')
plt.show()

# Tìm số lượng cột NaN nhiều nhất
max_nan = nan_per_row.max()
max_nan_count = nan_counts.get(max_nan, 0)
max_nan_ratio = nan_ratios.get(max_nan, 0)

print(
    f"\nThe row with the most NaN columns: {max_nan} NaN columns, with {max_nan_count} rows, ratio: {max_nan_ratio * 100:.2f}%")
# Remove rows with 17 NaN values
nan_per_row = combined_df.isna().sum(axis=1)
cleaned_df = combined_df[nan_per_row != 17]
combined_df.rename(columns={"tr_status": "build_failed"}, inplace=True)
combined_df = combined_df[combined_df["build_failed"].isin(["passed", "failed"])].copy()
combined_df["build_failed"] = combined_df["build_failed"].map({"passed": 0, "failed": 1})

### 2.3. Các dòng có 1 và 3 cột NaN

#### 2.3.2. Lọc

In [ ]:
rows_with_1_nan = combined_df[nan_per_row == 1].copy()
rows_with_3_nan = combined_df[nan_per_row == 3].copy()


# Hàm để phân tích và hiển thị dưới dạng bảng
def analyze_nan_rows(df, num_nan):
    """
    Phân tích các dòng có `num_nam` cột NaN, in ra tên các cột đó và giá trị của 'tr_status' hoặc 'build_failed'.

    Args:
        df: DataFrame chứa dữ liệu.
        num_nan: Số lượng cột NaN cần trên 1 dòng.
    """
    if len(df) == 0:
        print(f"\nKhông có dòng nào có {num_nan} cột NaN.")
        return

    # Tạo cột mới chứa danh sách các cột có NaN
    nan_cols = df.isna()
    df['NaN Columns'] = nan_cols.apply(lambda row: tuple(col for col, is_nan in row.items() if is_nan), axis=1)

    status_col = 'build_failed'
    if status_col not in df.columns:
        print(f"\nKhông tìm thấy cột 'build_failed' trong dữ liệu.")
        return

    # Nhóm các dòng theo tập hợp các cột NaN giống nhau
    grouped = df.groupby('NaN Columns')

    table_data = []
    possible_statuses = [0, 1]  # Giá trị của build_failed: 0 (thành công), 1 (thất bại)

    for nan_cols_group, group in grouped:
        row_count = len(group)

        # Đếm phân bố giá trị của status_col
        status_counts = group[status_col].value_counts()

        # Tạo một hàng cho bảng
        row = {
            'NaN Columns': ', '.join(nan_cols_group),
            'Row Count': row_count
        }

        # Thêm các cột cho từng giá trị của status_col, điền 0 nếu không có giá trị
        for status in possible_statuses:
            row[f'Build {status}'] = status_counts.get(status, 0)

        table_data.append(row)

    # Tạo DataFrame từ table_data
    result_table = pd.DataFrame(table_data)

    # Sắp xếp lại cột
    columns_order = ['NaN Columns', 'Row Count'] + [f'Build {status}' for status in possible_statuses]
    result_table = result_table[columns_order]

    return result_table


#### 2.3.2. Phân tích

In [ ]:
analyze_nan_rows(rows_with_1_nan, 1)
analyze_nan_rows(rows_with_3_nan, 3)
table_1_nan = analyze_nan_rows(rows_with_1_nan, 1)
table_3_nan = analyze_nan_rows(rows_with_3_nan, 3)
print(table_1_nan)
print(table_3_nan)
print("\nPhân tích tương quan với build failure:")

columns_to_analyze = ['git_num_commits', 'gh_num_issue_comments', 'gh_num_pr_comments']


def calculate_failure_rate(df, col_name):
    """
    Tính tỷ lệ build failure cho một cột cụ thể trong DataFrame.

    Args:
        df: DataFrame chứa dữ liệu.
        col_name: Tên cột cần phân tích.

    Returns:
        float: Tỷ lệ build failure (%).
    """
    total = len(df)
    if total == 0:
        return 0
    failed = len(df[df['build_failed'] == 1])
    return round(failed / total * 100, 2)


# Tạo danh sách để lưu dữ liệu bảng
table_data = []

# Phân tích có/không NaN
for col in columns_to_analyze:
    # Nhóm có NaN
    rows_with_nan = combined_df[combined_df[col].isna()]
    failure_rate_with_nan = calculate_failure_rate(rows_with_nan, col)
    rows_with_nan_count = len(rows_with_nan)

    # Nhóm không có NaN
    rows_without_nan = combined_df[combined_df[col].notna()]
    failure_rate_without_nan = calculate_failure_rate(rows_without_nan, col)
    rows_without_nan_count = len(rows_without_nan)

    # Tính hệ số tương quan (chỉ với các dòng không có NaN)
    correlation = None
    if len(rows_without_nan) > 0:
        correlation = rows_without_nan[[col, 'build_failed']].corr().iloc[0, 1]

    # Thêm hàng vào bảng
    table_data.append({
        'Column': col,
        'Rows with NaN': rows_with_nan_count,
        'Failure Rate (NaN)': failure_rate_with_nan,
        'Rows without NaN': rows_without_nan_count,
        'Failure Rate (Not NaN)': failure_rate_without_nan,
        'Correlation': correlation
    })

# Tạo DataFrame từ table_data
result_table = pd.DataFrame(table_data)

# Định dạng cột Correlation
result_table['Correlation'] = result_table['Correlation'].apply(lambda x: f"{x:.4f}" if x is not None else "N/A")

# In bảng
print("\nBảng phân tích tương quan với build failure:")
print(result_table)

# Vẽ biểu đồ so sánh tỷ lệ build failure
failure_rates = []
labels = []
for col in columns_to_analyze:
    rows_with_nan = combined_df[combined_df[col].isna()]
    rows_without_nan = combined_df[combined_df[col].notna()]
    failure_rates.extend([
        calculate_failure_rate(rows_with_nan, col),
        calculate_failure_rate(rows_without_nan, col)
    ])
    labels.extend([f"{col} (NaN)", f"{col} (Not NaN)"])

plt.figure(figsize=(6, 4))
plt.bar(labels, failure_rates, color=['#ff9999', '#66b3ff'] * len(columns_to_analyze))
plt.xlabel('Cột và trạng thái NaN')
plt.ylabel('Tỷ lệ build failure (%)')
plt.title('Tỷ lệ build failure: Có NaN vs Không NaN')
plt.xticks(rotation=45)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 3. Data Processing

### 3.1. Preprocess

In [ ]:
# Replace NaN values with default values
columns_required = ["build_failed", "gh_project_name",
                    "gh_build_started_at"]  # Only columns can not fill with default values
before_drop = len(combined_df)
combined_df.fillna(0, inplace=True)
print(f"\nDropped {before_drop - len(combined_df)} rows missing critical columns. New shape: {combined_df.shape}")

combined_df['gh_build_started_at'] = pd.to_datetime(combined_df['gh_build_started_at'])
# Group job by project_name, build_started_at, and build_failed
builds_df = combined_df.groupby([
    'gh_project_name',
    'gh_build_started_at',
    'build_failed'
], as_index=False).agg({
    'git_num_commits': 'sum',
    'gh_num_issue_comments': 'sum',
    'gh_num_pr_comments': 'sum',
    'gh_team_size': 'mean',
    'gh_sloc': 'mean',
    'git_diff_src_churn': 'sum',
    'git_diff_test_churn': 'sum',
    'gh_diff_files_added': 'sum',
    'gh_diff_files_deleted': 'sum',
    'gh_diff_files_modified': 'sum',
    'gh_diff_tests_added': 'sum',
    'gh_diff_tests_deleted': 'sum',
    'gh_diff_src_files': 'sum',
    'gh_diff_doc_files': 'sum',
    'gh_diff_other_files': 'sum',
    'gh_num_commits_on_files_touched': 'sum',
    'gh_test_lines_per_kloc': 'mean',
    'gh_test_cases_per_kloc': 'mean',
    'gh_asserts_cases_per_kloc': 'mean',
    'gh_is_pr': 'max',
    'gh_by_core_team_member': 'max',
    'gh_num_commit_comments': 'sum'
})
print(f"\nThe amount of data reduced after grouping: {combined_df.shape[0] - builds_df.shape[0]}")
print(f"\nShape after merging jobs into builds: {builds_df.shape}")

# Drop duplicates
num_duplicates = builds_df.duplicated().sum()
print(f"\nNumber of duplicates: {num_duplicates}")
initial_rows = len(builds_df)
builds_df.drop_duplicates(inplace=True)
print(f"\nDropped {initial_rows - len(builds_df)} duplicates. New shape: {builds_df.shape}")

# Check if DataFrame is empty after duplicates
if builds_df.empty:
    raise ValueError("DataFrame is empty after dropping duplicates. Check duplicate criteria or data quality.")

### 3.2. Convert data

In [ ]:
# Encode categorical columns
categorical_columns = ["gh_is_pr", "gh_by_core_team_member"]
label_encoders = {}
for col in categorical_columns:
    if col in builds_df.columns:
        le = LabelEncoder()
        builds_df[col] = le.fit_transform(builds_df[col].astype(str))
        label_encoders[col] = le
    else:
        print(f"Column {col} not found in the dataset. Skipping...")

# Normalize numerical columns
numerical_columns = [
    "git_num_commits",
    "gh_num_commit_comments",
    "git_diff_src_churn",
    "git_diff_test_churn",
    "gh_diff_files_added",
    "gh_diff_files_deleted",
    "gh_diff_files_modified",
    "gh_diff_tests_added",
    "gh_diff_tests_deleted",
    "gh_diff_src_files",
    "gh_diff_doc_files",
    "gh_diff_other_files",
    "gh_num_commits_on_files_touched",
    "gh_sloc",
    "gh_test_lines_per_kloc",
    "gh_test_cases_per_kloc",
    "gh_asserts_cases_per_kloc",
    "gh_team_size",
    "gh_num_issue_comments",
    "gh_num_pr_comments"
]
scaler = MinMaxScaler()
builds_df[numerical_columns] = scaler.fit_transform(builds_df[numerical_columns].fillna(0))
print("\nDataset after encoding and normalization:")
print(builds_df.head())
print(f"Final shape: {builds_df.shape}")
# Print imbalance information
imbalance = builds_df['build_failed'].value_counts(normalize=True)
print("Class distribution:")
print(imbalance)

# Plot class distribution
imbalance.plot(kind='bar', title='Class Distribution (build_failed)')
plt.xlabel('Class (0: Passed, 1: Failed)')
plt.ylabel('Proportion')
plt.show()

project_counts = builds_df['gh_project_name'].value_counts()
# Calculate class distribution for each project
balance_data = []
for project in project_counts.index:
    project_df = builds_df[builds_df['gh_project_name'] == project]
    class_counts = project_df['build_failed'].value_counts(normalize=True)
    failed_ratio = class_counts.get(1.0, 0.0)
    passed_ratio = class_counts.get(0.0, 0.0)
    if 0.2 <= passed_ratio <= 0.8 and 0.2 <= failed_ratio <= 0.8:
        balance_data.append({
            'project': project,
            'failed_ratio': failed_ratio,
            'passed_ratio': passed_ratio,
            'total_rows': len(project_df)
        })

# Create DataFrame and sort by failed ratio
balance_df = pd.DataFrame(balance_data)
balance_df = balance_df.sort_values(by='total_rows', ascending=False).head(10)

print("Top 10 Projects with Balanced Class Distribution (30%-70%):")
print(balance_df[['project', 'failed_ratio', 'passed_ratio', 'total_rows']])

# Plot top 10 projects by number of rows
plt.figure(figsize=(12, 6))
plt.bar(balance_df['project'], balance_df['total_rows'], color='lightblue')
plt.title('Top 10 Projects by Number of Rows (Balanced Class Distribution)')
plt.xlabel('Project Name')
plt.ylabel('Number of Rows')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Plot class distribution for top 10 projects
plt.figure(figsize=(14, 6))
bar_width = 0.4
x = np.arange(len(balance_df))

plt.bar(x - bar_width / 2, balance_df['passed_ratio'], width=bar_width, label='Passed (0)', color='skyblue')
plt.bar(x + bar_width / 2, balance_df['failed_ratio'], width=bar_width, label='Failed (1)', color='salmon')

plt.xticks(ticks=x, labels=balance_df['project'], rotation=45, ha='right')
plt.xlabel('Project Name')
plt.ylabel('Class Ratio')
plt.title('Class Distribution in Top 10 Projects (Balanced 30%-70%)')
plt.legend()
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Print class distribution for top 10 projects
print(balance_df[['project', 'failed_ratio', 'passed_ratio', 'total_rows']].sort_index())

## 4. Save data by project

In [ ]:
output_data_dir = "../data/processed"
top_projects = balance_df['project']
for project in top_projects:
    project_df = builds_df[builds_df['gh_project_name'] == project]
    file_name = project.replace("/", "_").replace(":", "_").replace(" ", "_") + ".csv"
    project_df.to_csv(os.path.join(output_data_dir, file_name), index=False)
    print(f"Saved {file_name} with {len(project_df)} rows")